1) Data Extraction

In [1]:
import numpy as np 
import pandas as pd

In [2]:
from bs4 import BeautifulSoup
import requests
heading=[]
article=[]
url = "https://www.news18.com/covid19-analytics-centre/"
print(url)
body = requests.get(url)
print(body)



https://www.news18.com/covid19-analytics-centre/
<Response [200]>


In [3]:
soup = BeautifulSoup(body.content, 'html.parser')
#article.append(soup.find("h1", class_=["entry-title","tdb-title-text"]).text.strip())
print ('Classes of each table:')
for table in soup.find_all ('table'):
  print (table.get ('class'))

Classes of each table:
['analytics-table']


In [4]:
heading=[]
a_table=soup.find("table","analytics-table")
thead=a_table.find("thead")
for div in thead.find_all("div","text"):
  #print(div)
  heading.append(div.text)
print(heading)

['State/UT', 'Total', 'Active', 'Discharged', 'Deaths']


In [5]:
a_table=soup.find("table","analytics-table")
tbody=a_table.find("tbody")

In [6]:
row= a_table.find_all('tr')
#print(row)
print(len(row))
cols = a_table.find_all('td')
print(len(cols))

37
180


In [7]:
state=[]
total=[]
active=[]
discharged=[]
death=[BlockingIOError]
for row in tbody.find_all('tr'):
        cols = row.find_all('td')
        if len(cols) == len(heading):
            state.append((cols[0].text.strip()))
            total.append(int(cols[1]['data-number'].strip()))
            active.append(int(cols[2]['data-number'].strip()))
            discharged.append(int(cols[3]['data-number'].strip()))
            death.append(int(cols[4]['data-number'].strip()))
death.pop(0)
death

[148407,
 71502,
 40303,
 38049,
 14733,
 23632,
 21531,
 26518,
 9205,
 9653,
 11043,
 14146,
 10714,
 10776,
 12302,
 4111,
 19289,
 8035,
 4785,
 7751,
 5331,
 4213,
 4013,
 726,
 1975,
 2149,
 940,
 1181,
 1624,
 296,
 499,
 782,
 231,
 4,
 52,
 129]

2)Dataframe Creation

In [8]:
df = pd.DataFrame(list(zip(state,total,active,discharged,death)),
               columns =heading)
df.head(10)
#df.info()

,State/UT,Total,Active,Discharged,Deaths
0,Maharashtra,8135977,293,7987277,148407
1,Kerala,6826658,1583,6753573,71502
2,Karnataka,4071388,1639,4029446,40303
3,Tamil Nadu,3594198,127,3556022,38049
4,Andhra Pradesh,2339058,10,2324315,14733
5,Uttar Pradesh,2128042,111,2104299,23632
6,West Bengal,2118494,53,2096910,21531
7,Delhi,2006996,30,1980448,26518
8,Odisha,1336467,81,1327181,9205
9,Rajasthan,1315274,66,1305555,9653


3)CSV file creation

In [9]:
df.to_csv('covid_output.csv')

4)Saving in database

In [10]:
!pip install pymongo
import pandas as pd
import pymongo
from pymongo import MongoClient

!apt install mongodb >log
!service mongodb start

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


 * Starting database mongodb
   ...done.


In [11]:
# Making a Connection with MongoClient
client = MongoClient()
# database
db = client["Health_analytics"]
# collection
company= db["Covid_details"]

In [12]:
data=df
data.reset_index(inplace=True)
data_dict = data.to_dict("records")
company.insert_one({"index":"Covid","data":data_dict})

In [13]:
data_from_db = company.find_one({"index":"Covid"})
df1 = pd.DataFrame(data_from_db["data"])
#df.set_index("Date",inplace=True)
print(df1)

    level_0  index             State/UT    Total  Active  Discharged  Deaths
0         0      0          Maharashtra  8135977     293     7987277  148407
1         1      1               Kerala  6826658    1583     6753573   71502
2         2      2            Karnataka  4071388    1639     4029446   40303
3         3      3           Tamil Nadu  3594198     127     3556022   38049
4         4      4       Andhra Pradesh  2339058      10     2324315   14733
5         5      5        Uttar Pradesh  2128042     111     2104299   23632
6         6      6          West Bengal  2118494      53     2096910   21531
7         7      7                Delhi  2006996      30     1980448   26518
8         8      8               Odisha  1336467      81     1327181    9205
9         9      9            Rajasthan  1315274      66     1305555    9653
10       10     10              Gujarat  1277464     186     1266235   11043
11       11     11         Chhattisgarh  1177733       7     1163580   14146

5)Analysis using Plotly

In [14]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import plot, iplot, init_notebook_mode

In [15]:
import plotly.express as px
import numpy

In [16]:
df.columns

Index(['index', 'State/UT', 'Total', 'Active', 'Discharged', 'Deaths'], dtype='object')

In [17]:
from plotly.subplots import make_subplots

In [18]:
state = 'State/UT'
cases = 'Total'
active='Active'
death='Deaths'
discharged='Discharged'
trace1 = go.Bar(
    x=df[state],
    y=df[cases],
    
)
trace2 = go.Scatter(
    x=df[state],
    y=df[death],
    #yaxis='y2'

)

trace3 = go.Scatter(
    x=df[state],
    y=df[active],
    )

trace4 = go.Pie(
    labels=df[state],
    values=df[active],
    domain=dict(x=[0, 0.5]),
     )


  
fig = make_subplots(
   rows=2, cols=2,
   specs=[[{"type": "bar"},{"type":"scatter"}],
          [{"type": "pie"},{"type": "pie"}]],
  subplot_titles=("Plot 1-Total cases in each state", "Plot 2-State wise death", "Plot 3-State wise active cases")

)

#fig = make_subplots(rows=2, cols=2, subplot_titles=("Plot 1-Total cases in each state", "Plot 2-State wise death", "Plot 3-State wise active cases"))
fig.add_trace(trace1, row=1, col=1)
fig.add_trace(trace2, row=1, col=2)
#fig.add_trace(trace3, row=2, col=1)
fig.add_trace(trace4, row=2, col=1)

fig.update_layout(height=1200, width=700,
                  title_text="Covid cases Analysis -INDIA")

#fig.update_layout(xaxis=dict(tickangle=45))
fig.show()

6)Saving image as png

In [20]:

import plotly.io as pio

In [21]:
!pip install -U kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
import os
if not os.path.exists("images"):
    os.mkdir("images")

In [24]:
fig.to_image(format="png", engine="kaleido") 
fig.write_image("images/fig1.png")
pio.write_image(fig, "covid_output.png")